# Portfolio Metrics and Performance Analysis

This notebook demonstrates comprehensive portfolio analysis capabilities from the `portfoliometrics.py` module, covering performance metrics, risk analysis, and Modern Portfolio Theory applications.

## Features Covered:
- Performance Metrics (Sharpe, Sortino, Information Ratio)
- Portfolio Return and Risk Calculations
- CAPM Analysis and Beta Calculations
- Modern Portfolio Theory (Efficient Frontier)
- Risk Attribution and Factor Analysis
- Portfolio Optimization and Rebalancing

In [ ]:
# Import necessary libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Import our portfoliometrics module
from portfoliometrics import (
    PortfolioConfig,
    PerformanceCalculator,
    PortfolioAnalyzer,
    CAPMAnalyzer,
    ModernPortfolioTheory,
    RiskAttributionAnalyzer,
    ComprehensivePortfolioFramework
)

print("✅ Successfully imported portfolio analysis components")
print("📊 Ready to analyze portfolio performance, risk, and optimization")

## 1. Configuration and Setup

Initialize the portfolio analysis framework with appropriate parameters.

In [ ]:
# Initialize portfolio configuration
config = PortfolioConfig(
    risk_free_rate=0.02,        # 2% risk-free rate
    benchmark_return=0.08,      # 8% benchmark return
    target_return=0.10,         # 10% target return
    rebalancing_frequency='quarterly'  # Quarterly rebalancing
)

# Create the comprehensive portfolio framework
portfolio_framework = ComprehensivePortfolioFramework(config)

print(f"Portfolio Analysis Framework Initialized:")
print(f"• Risk-Free Rate: {config.risk_free_rate:.2%}")
print(f"• Benchmark Return: {config.benchmark_return:.2%}")
print(f"• Target Return: {config.target_return:.2%}")
print(f"• Rebalancing Frequency: {config.rebalancing_frequency}")
print(f"• Max Weight per Asset: {config.max_weight:.1%}")
print(f"• Min Weight per Asset: {config.min_weight:.1%}")

## 2. Generate Sample Portfolio Data

Create realistic sample data for portfolio analysis demonstrations.

In [ ]:
# Generate sample asset returns
def generate_sample_portfolio_data(n_assets=6, n_periods=1000):
    """
    Generate sample asset returns with realistic correlations and properties.
    """
    # Asset parameters
    asset_names = ['US_Equity', 'EU_Equity', 'EM_Equity', 'US_Bonds', 'REITS', 'Commodities']
    
    # Expected annual returns (converted to daily)
    annual_returns = np.array([0.10, 0.08, 0.12, 0.04, 0.09, 0.06])
    daily_returns = annual_returns / 252
    
    # Annual volatilities (converted to daily)
    annual_vols = np.array([0.16, 0.18, 0.25, 0.05, 0.20, 0.22])
    daily_vols = annual_vols / np.sqrt(252)
    
    # Correlation matrix (realistic asset correlations)
    correlation_matrix = np.array([
        [1.00, 0.75, 0.60, 0.20, 0.65, 0.30],  # US Equity
        [0.75, 1.00, 0.70, 0.25, 0.55, 0.25],  # EU Equity
        [0.60, 0.70, 1.00, 0.15, 0.50, 0.40],  # EM Equity
        [0.20, 0.25, 0.15, 1.00, 0.30, -0.10], # US Bonds
        [0.65, 0.55, 0.50, 0.30, 1.00, 0.35],  # REITs
        [0.30, 0.25, 0.40, -0.10, 0.35, 1.00]  # Commodities
    ])
    
    # Generate correlated returns
    # Convert correlation to covariance matrix
    cov_matrix = np.outer(daily_vols, daily_vols) * correlation_matrix
    
    # Generate random returns
    returns = np.random.multivariate_normal(daily_returns, cov_matrix, n_periods)
    
    # Create DataFrame
    dates = pd.date_range(start='2020-01-01', periods=n_periods, freq='D')
    returns_df = pd.DataFrame(returns, columns=asset_names, index=dates)
    
    # Generate market benchmark (broad market index)
    market_weights = np.array([0.5, 0.2, 0.1, 0.15, 0.05, 0.0])  # Market cap weighted
    market_returns = (returns_df * market_weights).sum(axis=1)
    returns_df['Market_Benchmark'] = market_returns
    
    return returns_df

# Generate sample portfolio weights over time
def generate_portfolio_weights_timeseries(returns_df, rebalancing_freq='quarterly'):
    """
    Generate portfolio weights that change over time with rebalancing.
    """
    asset_names = returns_df.columns[:-1]  # Exclude benchmark
    n_assets = len(asset_names)
    
    # Strategic weights (long-term allocation)
    strategic_weights = np.array([0.25, 0.20, 0.15, 0.25, 0.10, 0.05])
    
    # Create rebalancing dates
    if rebalancing_freq == 'quarterly':
        rebal_freq = 63  # ~3 months
    elif rebalancing_freq == 'monthly':
        rebal_freq = 21  # ~1 month
    else:
        rebal_freq = 252  # Annual
    
    weights_df = pd.DataFrame(index=returns_df.index, columns=asset_names)
    
    # Initialize weights
    current_weights = strategic_weights.copy()
    
    for i, date in enumerate(returns_df.index):
        if i % rebal_freq == 0:
            # Rebalancing: slight drift from strategic weights
            drift = np.random.normal(0, 0.02, n_assets)  # 2% random drift
            current_weights = strategic_weights + drift
            current_weights = np.maximum(current_weights, 0)  # No negative weights
            current_weights = current_weights / current_weights.sum()  # Normalize
        else:
            # Natural drift due to performance
            if i > 0:
                daily_returns = returns_df.iloc[i, :-1].values
                current_weights = current_weights * (1 + daily_returns)
                current_weights = current_weights / current_weights.sum()
        
        weights_df.iloc[i] = current_weights
    
    return weights_df

# Generate the sample data
returns_data = generate_sample_portfolio_data()
weights_data = generate_portfolio_weights_timeseries(returns_data)

print(f"📊 Generated Sample Data:")
print(f"• Returns Data: {returns_data.shape[0]} days × {returns_data.shape[1]} assets")
print(f"• Weights Data: {weights_data.shape[0]} days × {weights_data.shape[1]} assets")
print(f"• Date Range: {returns_data.index[0].date()} to {returns_data.index[-1].date()}")

# Display asset statistics
print(f"\n📈 Annual Asset Statistics:")
annual_stats = pd.DataFrame({
    'Annual_Return': returns_data.iloc[:, :-1].mean() * 252,
    'Annual_Volatility': returns_data.iloc[:, :-1].std() * np.sqrt(252),
    'Sharpe_Ratio': (returns_data.iloc[:, :-1].mean() * 252 - config.risk_free_rate) / 
                   (returns_data.iloc[:, :-1].std() * np.sqrt(252))
})
print(annual_stats.round(3))

## 3. Performance Metrics Analysis

Calculate comprehensive performance metrics for the portfolio.

In [ ]:
# Initialize performance calculator
performance_calc = PerformanceCalculator(config)

# Calculate portfolio returns
portfolio_returns = (returns_data.iloc[:, :-1] * weights_data).sum(axis=1)
benchmark_returns = returns_data['Market_Benchmark']

print(f"📊 Performance Metrics Analysis:")
print("=" * 45)

# 1. Basic Return Metrics
return_metrics = performance_calc.calculate_return_metrics(
    portfolio_returns, benchmark_returns
)

print(f"📈 Return Metrics:")
print(f"• Portfolio Annual Return: {return_metrics['portfolio_annual_return']:.2%}")
print(f"• Benchmark Annual Return: {return_metrics['benchmark_annual_return']:.2%}")
print(f"• Excess Return: {return_metrics['excess_return']:.2%}")
print(f"• Cumulative Return: {return_metrics['cumulative_return']:.2%}")
print(f"• Annualized Return: {return_metrics['annualized_return']:.2%}")

# 2. Risk Metrics
risk_metrics = performance_calc.calculate_risk_metrics(
    portfolio_returns, benchmark_returns
)

print(f"\n⚠️ Risk Metrics:")
print(f"• Portfolio Volatility: {risk_metrics['portfolio_volatility']:.2%}")
print(f"• Benchmark Volatility: {risk_metrics['benchmark_volatility']:.2%}")
print(f"• Tracking Error: {risk_metrics['tracking_error']:.2%}")
print(f"• Downside Deviation: {risk_metrics['downside_deviation']:.2%}")
print(f"• Maximum Drawdown: {risk_metrics['max_drawdown']:.2%}")
print(f"• VaR (95%): {risk_metrics['var_95']:.2%}")

# 3. Risk-Adjusted Performance
risk_adj_metrics = performance_calc.calculate_risk_adjusted_metrics(
    portfolio_returns, benchmark_returns, config.risk_free_rate
)

print(f"\n🎯 Risk-Adjusted Performance:")
print(f"• Sharpe Ratio: {risk_adj_metrics['sharpe_ratio']:.3f}")
print(f"• Sortino Ratio: {risk_adj_metrics['sortino_ratio']:.3f}")
print(f"• Information Ratio: {risk_adj_metrics['information_ratio']:.3f}")
print(f"• Calmar Ratio: {risk_adj_metrics['calmar_ratio']:.3f}")
print(f"• Treynor Ratio: {risk_adj_metrics['treynor_ratio']:.3f}")

# 4. Additional Performance Metrics
portfolio_stats = performance_calc.portfolio_performance_statistics(
    portfolio_returns, benchmark_returns
)

print(f"\n📊 Additional Statistics:")
print(f"• Alpha (Jensen's): {portfolio_stats['alpha']:.3f}")
print(f"• Beta: {portfolio_stats['beta']:.3f}")
print(f"• R-squared: {portfolio_stats['r_squared']:.3f}")
print(f"• Skewness: {portfolio_stats['skewness']:.3f}")
print(f"• Kurtosis: {portfolio_stats['kurtosis']:.3f}")
print(f"• Win Rate: {portfolio_stats['win_rate']:.1%}")
print(f"• Best Month: {portfolio_stats['best_month']:.2%}")
print(f"• Worst Month: {portfolio_stats['worst_month']:.2%}")

# 5. Rolling Performance Analysis
rolling_metrics = performance_calc.rolling_performance_analysis(
    portfolio_returns, benchmark_returns, window=252  # 1-year rolling
)

print(f"\n📊 Rolling 1-Year Performance (Current):")
current_sharpe = rolling_metrics['rolling_sharpe'].iloc[-1]
current_alpha = rolling_metrics['rolling_alpha'].iloc[-1]
current_beta = rolling_metrics['rolling_beta'].iloc[-1]

print(f"• Current Rolling Sharpe: {current_sharpe:.3f}")
print(f"• Current Rolling Alpha: {current_alpha:.3f}")
print(f"• Current Rolling Beta: {current_beta:.3f}")
print(f"• Rolling Sharpe Range: {rolling_metrics['rolling_sharpe'].min():.3f} to {rolling_metrics['rolling_sharpe'].max():.3f}")

# 6. Drawdown Analysis
drawdown_analysis = performance_calc.drawdown_analysis(portfolio_returns)

print(f"\n📉 Drawdown Analysis:")
print(f"• Maximum Drawdown: {drawdown_analysis['max_drawdown']:.2%}")
print(f"• Drawdown Duration: {drawdown_analysis['max_drawdown_duration']} days")
print(f"• Recovery Time: {drawdown_analysis['recovery_time']} days")
print(f"• Current Drawdown: {drawdown_analysis['current_drawdown']:.2%}")
print(f"• Avg Drawdown: {drawdown_analysis['average_drawdown']:.2%}")

## 4. Portfolio Risk and Return Analysis

Analyze portfolio construction, risk decomposition, and contribution analysis.

In [ ]:
# Initialize portfolio analyzer
portfolio_analyzer = PortfolioAnalyzer(config)

print(f"📊 Portfolio Analysis:")
print("=" * 30)

# 1. Portfolio Risk Decomposition
current_weights = weights_data.iloc[-1].values  # Most recent weights
asset_returns = returns_data.iloc[:, :-1]  # Exclude benchmark

risk_decomp = portfolio_analyzer.portfolio_risk_decomposition(
    asset_returns, current_weights
)

print(f"📊 Portfolio Risk Decomposition:")
print(f"• Portfolio Variance: {risk_decomp['portfolio_variance']:.6f}")
print(f"• Portfolio Volatility: {risk_decomp['portfolio_volatility']:.3%}")
print(f"• Annualized Volatility: {risk_decomp['annualized_volatility']:.2%}")

print(f"\n🧩 Risk Contributions by Asset:")
for asset, contribution in risk_decomp['risk_contributions'].items():
    weight = current_weights[list(asset_returns.columns).index(asset)]
    print(f"• {asset}: {contribution:.3%} (Weight: {weight:.1%})")

# 2. Portfolio Return Attribution
return_attribution = portfolio_analyzer.portfolio_return_attribution(
    asset_returns, weights_data
)

print(f"\n💰 Return Attribution (Last Period):")
total_return = return_attribution['total_return'].iloc[-1]
print(f"• Total Portfolio Return: {total_return:.3%}")

print(f"\nReturn Contributions by Asset:")
for asset in asset_returns.columns:
    contribution = return_attribution['return_contributions'][asset].iloc[-1]
    print(f"• {asset}: {contribution:.3%}")

# 3. Diversification Analysis
diversification = portfolio_analyzer.diversification_analysis(
    asset_returns, current_weights
)

print(f"\n🌐 Diversification Analysis:")
print(f"• Diversification Ratio: {diversification['diversification_ratio']:.3f}")
print(f"• Effective Number of Assets: {diversification['effective_n_assets']:.2f}")
print(f"• Concentration Index (HHI): {diversification['concentration_index']:.3f}")
print(f"• Maximum Weight: {diversification['max_weight']:.1%}")
print(f"• Diversification Benefit: {diversification['diversification_benefit']:.3%}")

# 4. Correlation Analysis
correlation_analysis = portfolio_analyzer.correlation_analysis(
    asset_returns, current_weights
)

print(f"\n🔗 Correlation Analysis:")
print(f"• Average Correlation: {correlation_analysis['average_correlation']:.3f}")
print(f"• Weighted Average Correlation: {correlation_analysis['weighted_avg_correlation']:.3f}")
print(f"• Maximum Correlation: {correlation_analysis['max_correlation']:.3f}")
print(f"• Minimum Correlation: {correlation_analysis['min_correlation']:.3f}")

print(f"\nHighest Correlated Pairs:")
corr_matrix = asset_returns.corr()
# Get top 3 correlations (excluding self-correlations)
correlations = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        correlations.append((
            corr_matrix.columns[i], 
            corr_matrix.columns[j], 
            corr_matrix.iloc[i, j]
        ))

correlations.sort(key=lambda x: abs(x[2]), reverse=True)
for asset1, asset2, corr in correlations[:3]:
    print(f"• {asset1} - {asset2}: {corr:.3f}")

# 5. Rebalancing Analysis
rebalancing = portfolio_analyzer.rebalancing_analysis(
    weights_data, target_weights=weights_data.iloc[0].values
)

print(f"\n⚖️ Rebalancing Analysis:")
print(f"• Average Turnover: {rebalancing['average_turnover']:.2%}")
print(f"• Maximum Drift: {rebalancing['max_drift']:.2%}")
print(f"• Rebalancing Frequency: {rebalancing['rebalancing_frequency']}")
print(f"• Transaction Costs: {rebalancing['estimated_transaction_costs']:.3%}")

# 6. Risk Budget Analysis
risk_budget = portfolio_analyzer.risk_budget_analysis(
    asset_returns, current_weights, target_risk_contributions=current_weights
)

print(f"\n📊 Risk Budget Analysis:")
print(f"• Total Risk Budget Used: {sum(risk_budget['actual_risk_contributions'].values()):.1%}")

print(f"\nRisk Budget vs Actual:")
for asset in asset_returns.columns:
    target = risk_budget['target_risk_contributions'][asset]
    actual = risk_budget['actual_risk_contributions'][asset]
    deviation = actual - target
    print(f"• {asset}: Target {target:.1%}, Actual {actual:.1%}, Deviation {deviation:+.1%}")

## 5. CAPM Analysis and Beta Calculations

Perform Capital Asset Pricing Model analysis and calculate various beta measures.

In [ ]:
# Initialize CAPM analyzer
capm_analyzer = CAPMAnalyzer(config)

print(f"📊 CAPM Analysis:")
print("=" * 25)

# 1. Portfolio Beta Analysis
portfolio_beta = capm_analyzer.calculate_portfolio_beta(
    portfolio_returns, benchmark_returns
)

print(f"📈 Portfolio Beta Analysis:")
print(f"• Portfolio Beta: {portfolio_beta['beta']:.3f}")
print(f"• Alpha: {portfolio_beta['alpha']:.4f}")
print(f"• R-squared: {portfolio_beta['r_squared']:.3f}")
print(f"• Standard Error: {portfolio_beta['std_error']:.4f}")
print(f"• t-statistic: {portfolio_beta['t_statistic']:.3f}")
print(f"• p-value: {portfolio_beta['p_value']:.4f}")

# Beta interpretation
beta_val = portfolio_beta['beta']
if beta_val > 1.2:
    beta_desc = "High Beta (Aggressive)"
elif beta_val > 0.8:
    beta_desc = "Moderate Beta"
else:
    beta_desc = "Low Beta (Defensive)"

print(f"• Beta Classification: {beta_desc}")

# 2. Individual Asset Betas
print(f"\n📊 Individual Asset Betas:")
asset_betas = {}
for asset in asset_returns.columns:
    asset_beta = capm_analyzer.calculate_stock_beta_variance_covariance(
        asset_returns[asset], benchmark_returns
    )
    asset_betas[asset] = asset_beta
    print(f"• {asset}: Beta = {asset_beta['beta']:.3f}, R² = {asset_beta['r_squared']:.3f}")

# 3. Alternative Beta Calculations
print(f"\n🔄 Alternative Beta Calculations (US_Equity):")

# Correlation method
corr_beta = capm_analyzer.calculate_stock_beta_correlation(
    asset_returns['US_Equity'], benchmark_returns
)
print(f"• Correlation Method Beta: {corr_beta['beta']:.3f}")
print(f"  - Correlation: {corr_beta['correlation']:.3f}")
print(f"  - Stock Volatility: {corr_beta['stock_volatility']:.3%}")
print(f"  - Market Volatility: {corr_beta['market_volatility']:.3%}")

# Rolling beta
rolling_beta = capm_analyzer.calculate_rolling_beta(
    asset_returns['US_Equity'], benchmark_returns, window=252
)
print(f"\n• Rolling Beta (1-year window):")
print(f"  - Current Beta: {rolling_beta['rolling_beta'].iloc[-1]:.3f}")
print(f"  - Average Beta: {rolling_beta['rolling_beta'].mean():.3f}")
print(f"  - Beta Volatility: {rolling_beta['rolling_beta'].std():.3f}")
print(f"  - Min/Max Beta: {rolling_beta['rolling_beta'].min():.3f} / {rolling_beta['rolling_beta'].max():.3f}")

# 4. CAPM Expected Returns
print(f"\n💰 CAPM Expected Returns:")
for asset, beta_info in asset_betas.items():
    expected_return = capm_analyzer.capm_expected_return(
        config.risk_free_rate, beta_info['beta'], config.benchmark_return
    )
    actual_return = asset_returns[asset].mean() * 252  # Annualized
    
    print(f"• {asset}:")
    print(f"  - CAPM Expected: {expected_return:.2%}")
    print(f"  - Actual Return: {actual_return:.2%}")
    print(f"  - Alpha: {actual_return - expected_return:+.2%}")

# 5. Security Market Line Analysis
sml_analysis = capm_analyzer.security_market_line_analysis(
    asset_returns, benchmark_returns, config.risk_free_rate
)

print(f"\n📈 Security Market Line Analysis:")
print(f"• Market Risk Premium: {sml_analysis['market_risk_premium']:.2%}")
print(f"• SML Slope: {sml_analysis['sml_slope']:.4f}")

print(f"\nAsset Positioning vs SML:")
for asset, position in sml_analysis['asset_positions'].items():
    if position['alpha'] > 0.01:  # 1% threshold
        status = "Above SML (Outperforming)"
    elif position['alpha'] < -0.01:
        status = "Below SML (Underperforming)"
    else:
        status = "On SML (Fair Value)"
    
    print(f"• {asset}: {status} (Alpha: {position['alpha']:+.2%})")

# 6. Multi-factor Beta Analysis (Fama-French style)
print(f"\n🔍 Multi-Factor Analysis:")

# Create style factors (simplified)
size_factor = asset_returns['EM_Equity'] - asset_returns['US_Equity']  # SMB proxy
value_factor = asset_returns['US_Bonds'] - asset_returns['US_Equity']  # HML proxy

multi_factor = capm_analyzer.multi_factor_model(
    portfolio_returns, 
    {
        'Market': benchmark_returns,
        'Size': size_factor,
        'Value': value_factor
    }
)

print(f"• Market Beta: {multi_factor['factor_loadings']['Market']:.3f}")
print(f"• Size Beta: {multi_factor['factor_loadings']['Size']:.3f}")
print(f"• Value Beta: {multi_factor['factor_loadings']['Value']:.3f}")
print(f"• Multi-factor Alpha: {multi_factor['alpha']:.4f}")
print(f"• Multi-factor R²: {multi_factor['r_squared']:.3f}")
print(f"• Improvement over CAPM: {multi_factor['r_squared'] - portfolio_beta['r_squared']:+.3f}")

## 6. Modern Portfolio Theory and Optimization

Apply Modern Portfolio Theory to construct efficient portfolios and analyze the efficient frontier.

In [ ]:
# Initialize Modern Portfolio Theory analyzer
mpt_analyzer = ModernPortfolioTheory(config)

print(f"📊 Modern Portfolio Theory Analysis:")
print("=" * 40)

# 1. Efficient Frontier Calculation
expected_returns = asset_returns.mean() * 252  # Annualized
covariance_matrix = asset_returns.cov() * 252  # Annualized

efficient_frontier = mpt_analyzer.calculate_efficient_frontier(
    expected_returns, covariance_matrix, num_portfolios=50
)

print(f"📈 Efficient Frontier Analysis:")
print(f"• Number of efficient portfolios: {len(efficient_frontier['returns'])}")
print(f"• Return range: {min(efficient_frontier['returns']):.2%} to {max(efficient_frontier['returns']):.2%}")
print(f"• Risk range: {min(efficient_frontier['volatilities']):.2%} to {max(efficient_frontier['volatilities']):.2%}")
print(f"• Max Sharpe ratio: {max(efficient_frontier['sharpe_ratios']):.3f}")

# 2. Optimal Portfolios
# Minimum Variance Portfolio
min_var_portfolio = mpt_analyzer.minimum_variance_portfolio(
    covariance_matrix
)

print(f"\n📊 Minimum Variance Portfolio:")
print(f"• Expected Return: {min_var_portfolio['expected_return']:.2%}")
print(f"• Volatility: {min_var_portfolio['volatility']:.2%}")
print(f"• Sharpe Ratio: {min_var_portfolio['sharpe_ratio']:.3f}")

print(f"\nWeights:")
for asset, weight in min_var_portfolio['weights'].items():
    print(f"• {asset}: {weight:.1%}")

# Maximum Sharpe Portfolio
max_sharpe_portfolio = mpt_analyzer.maximum_sharpe_portfolio(
    expected_returns, covariance_matrix, config.risk_free_rate
)

print(f"\n🎯 Maximum Sharpe Portfolio:")
print(f"• Expected Return: {max_sharpe_portfolio['expected_return']:.2%}")
print(f"• Volatility: {max_sharpe_portfolio['volatility']:.2%}")
print(f"• Sharpe Ratio: {max_sharpe_portfolio['sharpe_ratio']:.3f}")

print(f"\nWeights:")
for asset, weight in max_sharpe_portfolio['weights'].items():
    print(f"• {asset}: {weight:.1%}")

# Target Return Portfolio
target_return_portfolio = mpt_analyzer.target_return_portfolio(
    expected_returns, covariance_matrix, target_return=0.10
)

print(f"\n🎯 Target Return Portfolio (10%):")
print(f"• Expected Return: {target_return_portfolio['expected_return']:.2%}")
print(f"• Volatility: {target_return_portfolio['volatility']:.2%}")
print(f"• Sharpe Ratio: {target_return_portfolio['sharpe_ratio']:.3f}")

# 3. Black-Litterman Model (simplified)
# Market capitalization weights (proxy)
market_caps = np.array([50, 30, 15, 80, 10, 5])  # Relative market caps
market_weights = market_caps / market_caps.sum()

# Investor views (simplified)
views = {
    'US_Equity': 0.08,   # Expect 8% return
    'EM_Equity': 0.15    # Expect 15% return
}
view_confidence = 0.25  # 25% confidence

bl_portfolio = mpt_analyzer.black_litterman_optimization(
    expected_returns, covariance_matrix, market_weights, views, view_confidence
)

print(f"\n🔮 Black-Litterman Portfolio:")
print(f"• Expected Return: {bl_portfolio['expected_return']:.2%}")
print(f"• Volatility: {bl_portfolio['volatility']:.2%}")
print(f"• Sharpe Ratio: {bl_portfolio['sharpe_ratio']:.3f}")

print(f"\nWeights vs Market:")
for i, asset in enumerate(asset_returns.columns):
    bl_weight = bl_portfolio['weights'][asset]
    mkt_weight = market_weights[i]
    print(f"• {asset}: BL {bl_weight:.1%}, Market {mkt_weight:.1%}, Diff {bl_weight-mkt_weight:+.1%}")

# 4. Risk Parity Portfolio
risk_parity = mpt_analyzer.risk_parity_portfolio(covariance_matrix)

print(f"\n⚖️ Risk Parity Portfolio:")
print(f"• Expected Return: {risk_parity['expected_return']:.2%}")
print(f"• Volatility: {risk_parity['volatility']:.2%}")
print(f"• Sharpe Ratio: {risk_parity['sharpe_ratio']:.3f}")

print(f"\nRisk Contributions (should be equal):")
for asset, risk_contrib in risk_parity['risk_contributions'].items():
    print(f"• {asset}: {risk_contrib:.1%}")

# 5. Portfolio Comparison
portfolios = {
    'Current': {
        'weights': dict(zip(asset_returns.columns, current_weights)),
        'return': (current_weights * expected_returns).sum(),
        'volatility': np.sqrt(np.dot(current_weights, np.dot(covariance_matrix, current_weights)))
    },
    'Min Variance': min_var_portfolio,
    'Max Sharpe': max_sharpe_portfolio,
    'Risk Parity': risk_parity,
    'Black-Litterman': bl_portfolio
}

print(f"\n📊 Portfolio Comparison Summary:")
print(f"{'Portfolio':<15} {'Return':<8} {'Risk':<8} {'Sharpe':<8}")
print("-" * 45)

for name, portfolio in portfolios.items():
    ret = portfolio['expected_return'] if 'expected_return' in portfolio else portfolio['return']
    vol = portfolio['volatility']
    sharpe = (ret - config.risk_free_rate) / vol
    print(f"{name:<15} {ret:<8.2%} {vol:<8.2%} {sharpe:<8.3f}")

# 6. Resampled Efficient Frontier (Michaud)
resampled_frontier = mpt_analyzer.resampled_efficient_frontier(
    asset_returns, num_simulations=100, num_portfolios=20
)

print(f"\n🔄 Resampled Efficient Frontier:")
print(f"• Number of simulations: {resampled_frontier['num_simulations']}")
print(f"• Average Sharpe ratio: {np.mean(resampled_frontier['sharpe_ratios']):.3f}")
print(f"• Sharpe ratio std dev: {np.std(resampled_frontier['sharpe_ratios']):.3f}")
print(f"• Max resampled Sharpe: {max(resampled_frontier['sharpe_ratios']):.3f}")

## 7. Risk Attribution and Factor Analysis

Perform advanced risk attribution and factor-based analysis.

In [ ]:
# Initialize risk attribution analyzer
risk_attribution = RiskAttributionAnalyzer(config)

print(f"📊 Risk Attribution Analysis:")
print("=" * 35)

# 1. Factor Exposure Analysis
# Define style factors
factors = pd.DataFrame({
    'Market': benchmark_returns,
    'Size': asset_returns['EM_Equity'] - asset_returns['US_Equity'],  # Small minus Large
    'Value': asset_returns['US_Bonds'] - asset_returns['US_Equity'],  # High minus Low
    'Momentum': portfolio_returns.rolling(21).mean() - portfolio_returns.rolling(63).mean(),  # Momentum proxy
    'Quality': asset_returns['US_Bonds'] - asset_returns['Commodities']  # Quality proxy
})

factor_exposure = risk_attribution.factor_exposure_analysis(
    portfolio_returns, factors.dropna()
)

print(f"📊 Factor Exposure Analysis:")
print(f"• Model R-squared: {factor_exposure['r_squared']:.3f}")
print(f"• Adjusted R-squared: {factor_exposure['adj_r_squared']:.3f}")
print(f"• Residual Volatility: {factor_exposure['residual_volatility']:.3%}")

print(f"\nFactor Loadings:")
for factor, loading in factor_exposure['factor_loadings'].items():
    t_stat = factor_exposure['t_statistics'][factor]
    significance = "***" if abs(t_stat) > 2.58 else "**" if abs(t_stat) > 1.96 else "*" if abs(t_stat) > 1.64 else ""
    print(f"• {factor}: {loading:.3f} (t-stat: {t_stat:.2f}){significance}")

# 2. Risk Factor Decomposition
risk_decomp = risk_attribution.risk_factor_decomposition(
    portfolio_returns, factors.dropna()
)

print(f"\n📊 Risk Factor Decomposition:")
print(f"• Total Portfolio Variance: {risk_decomp['total_variance']:.6f}")
print(f"• Systematic Risk: {risk_decomp['systematic_risk']:.2%}")
print(f"• Idiosyncratic Risk: {risk_decomp['idiosyncratic_risk']:.2%}")

print(f"\nRisk Contribution by Factor:")
for factor, contribution in risk_decomp['factor_contributions'].items():
    percentage = contribution / risk_decomp['total_variance'] * 100
    print(f"• {factor}: {contribution:.6f} ({percentage:.1f}%)")

# 3. Style Analysis (Sharpe Style)
style_benchmarks = asset_returns.iloc[:, :4]  # Use first 4 assets as style benchmarks

style_analysis = risk_attribution.style_analysis(
    portfolio_returns, style_benchmarks
)

print(f"\n🎨 Style Analysis:")
print(f"• R-squared: {style_analysis['r_squared']:.3f}")
print(f"• Selection Return: {style_analysis['selection_return']:.3%}")
print(f"• Timing Return: {style_analysis['timing_return']:.3%}")

print(f"\nStyle Exposures:")
for style, exposure in style_analysis['style_weights'].items():
    print(f"• {style}: {exposure:.1%}")

# 4. Performance Attribution
# Use strategic weights as benchmark
benchmark_weights = np.array([0.25, 0.20, 0.15, 0.25, 0.10, 0.05])

perf_attribution = risk_attribution.performance_attribution(
    asset_returns, current_weights, benchmark_weights
)

print(f"\n💰 Performance Attribution:")
print(f"• Total Active Return: {perf_attribution['total_active_return']:.3%}")
print(f"• Asset Allocation Effect: {perf_attribution['allocation_effect']:.3%}")
print(f"• Security Selection Effect: {perf_attribution['selection_effect']:.3%}")
print(f"• Interaction Effect: {perf_attribution['interaction_effect']:.3%}")

print(f"\nAsset-Level Attribution:")
for asset in asset_returns.columns:
    alloc = perf_attribution['asset_allocation'][asset]
    select = perf_attribution['security_selection'][asset]
    total = alloc + select
    print(f"• {asset}: Total {total:+.3%} (Alloc: {alloc:+.3%}, Select: {select:+.3%})")

# 5. Tracking Error Decomposition
tracking_error_decomp = risk_attribution.tracking_error_decomposition(
    portfolio_returns, benchmark_returns, current_weights, benchmark_weights
)

print(f"\n📊 Tracking Error Decomposition:")
print(f"• Total Tracking Error: {tracking_error_decomp['total_tracking_error']:.3%}")
print(f"• Asset Allocation TE: {tracking_error_decomp['allocation_te']:.3%}")
print(f"• Security Selection TE: {tracking_error_decomp['selection_te']:.3%}")

print(f"\nTE Contribution by Asset:")
for asset, te_contrib in tracking_error_decomp['te_contributions'].items():
    print(f"• {asset}: {te_contrib:.3%}")

# 6. Sector/Geographic Attribution (simplified)
# Create sector mappings
sector_mapping = {
    'US_Equity': 'Developed Equity',
    'EU_Equity': 'Developed Equity', 
    'EM_Equity': 'Emerging Equity',
    'US_Bonds': 'Fixed Income',
    'REITS': 'Real Estate',
    'Commodities': 'Alternatives'
}

sector_attribution = risk_attribution.sector_attribution(
    asset_returns, current_weights, benchmark_weights, sector_mapping
)

print(f"\n🏭 Sector Attribution:")
print(f"• Total Active Return: {sector_attribution['total_active_return']:.3%}")

print(f"\nSector Contributions:")
for sector, contribution in sector_attribution['sector_contributions'].items():
    print(f"• {sector}: {contribution:.3%}")

# 7. Risk Model Validation
validation = risk_attribution.risk_model_validation(
    portfolio_returns, factors.dropna(), window=252
)

print(f"\n✅ Risk Model Validation:")
print(f"• Average R-squared: {validation['avg_r_squared']:.3f}")
print(f"• R-squared Stability: {validation['r_squared_stability']:.3f}")
print(f"• Forecast Accuracy: {validation['forecast_accuracy']:.3f}")
print(f"• Model Consistency: {'High' if validation['avg_r_squared'] > 0.7 else 'Medium' if validation['avg_r_squared'] > 0.5 else 'Low'}")

## 8. Portfolio Visualization Dashboard

Create comprehensive visualizations for portfolio analysis.

In [ ]:
# Create comprehensive portfolio dashboard
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
fig = plt.figure(figsize=(20, 16))

# 1. Cumulative Performance
ax1 = plt.subplot(3, 4, 1)
portfolio_cumulative = (1 + portfolio_returns).cumprod()
benchmark_cumulative = (1 + benchmark_returns).cumprod()

ax1.plot(portfolio_cumulative.index, portfolio_cumulative, 'b-', linewidth=2, label='Portfolio')
ax1.plot(benchmark_cumulative.index, benchmark_cumulative, 'r--', linewidth=2, label='Benchmark')
ax1.set_title('Cumulative Performance', fontweight='bold')
ax1.set_ylabel('Cumulative Return')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Rolling Sharpe Ratio
ax2 = plt.subplot(3, 4, 2)
ax2.plot(rolling_metrics['rolling_sharpe'].index, rolling_metrics['rolling_sharpe'], 'g-', linewidth=2)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax2.axhline(y=1, color='orange', linestyle='--', alpha=0.7, label='Good (>1.0)')
ax2.set_title('Rolling 1-Year Sharpe Ratio', fontweight='bold')
ax2.set_ylabel('Sharpe Ratio')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Portfolio Weights Over Time
ax3 = plt.subplot(3, 4, 3)
weights_data.plot(kind='area', stacked=True, ax=ax3, alpha=0.7)
ax3.set_title('Portfolio Weights Over Time', fontweight='bold')
ax3.set_ylabel('Weight')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# 4. Efficient Frontier
ax4 = plt.subplot(3, 4, 4)
ax4.scatter(efficient_frontier['volatilities'], efficient_frontier['returns'], 
           c=efficient_frontier['sharpe_ratios'], cmap='RdYlGn', alpha=0.7, s=20)

# Plot special portfolios
ax4.scatter(min_var_portfolio['volatility'], min_var_portfolio['expected_return'], 
           color='blue', marker='*', s=100, label='Min Variance')
ax4.scatter(max_sharpe_portfolio['volatility'], max_sharpe_portfolio['expected_return'], 
           color='red', marker='*', s=100, label='Max Sharpe')

# Current portfolio
current_return = (current_weights * expected_returns).sum()
current_vol = np.sqrt(np.dot(current_weights, np.dot(covariance_matrix, current_weights)))
ax4.scatter(current_vol, current_return, color='black', marker='o', s=100, label='Current')

ax4.set_title('Efficient Frontier', fontweight='bold')
ax4.set_xlabel('Volatility')
ax4.set_ylabel('Expected Return')
ax4.legend(fontsize=8)
ax4.grid(True, alpha=0.3)

# 5. Risk Contribution
ax5 = plt.subplot(3, 4, 5)
risk_contributions = list(risk_decomp['risk_contributions'].values())
asset_names = list(risk_decomp['risk_contributions'].keys())

bars = ax5.bar(asset_names, risk_contributions, alpha=0.7, color='lightcoral')
ax5.set_title('Risk Contribution by Asset', fontweight='bold')
ax5.set_ylabel('Risk Contribution')
ax5.tick_params(axis='x', rotation=45)

# Add percentage labels
total_risk = sum(risk_contributions)
for bar, contrib in zip(bars, risk_contributions):
    height = bar.get_height()
    ax5.text(bar.get_x() + bar.get_width()/2., height,
             f'{contrib/total_risk*100:.1f}%', ha='center', va='bottom', fontsize=8)

# 6. Asset Correlation Heatmap
ax6 = plt.subplot(3, 4, 6)
correlation_matrix = asset_returns.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
           square=True, ax=ax6, cbar_kws={'shrink': 0.8}, fmt='.2f')
ax6.set_title('Asset Correlation Matrix', fontweight='bold')

# 7. Drawdown Analysis
ax7 = plt.subplot(3, 4, 7)
drawdowns = drawdown_analysis['drawdown_series']
ax7.fill_between(drawdowns.index, 0, drawdowns, color='red', alpha=0.3)
ax7.plot(drawdowns.index, drawdowns, 'r-', linewidth=1)
ax7.set_title('Portfolio Drawdowns', fontweight='bold')
ax7.set_ylabel('Drawdown')
ax7.grid(True, alpha=0.3)

# 8. Factor Loadings
ax8 = plt.subplot(3, 4, 8)
factor_names = list(factor_exposure['factor_loadings'].keys())
factor_loadings = list(factor_exposure['factor_loadings'].values())
colors = ['green' if x > 0 else 'red' for x in factor_loadings]

bars = ax8.barh(factor_names, factor_loadings, color=colors, alpha=0.7)
ax8.axvline(x=0, color='black', linestyle='-', alpha=0.5)
ax8.set_title('Factor Loadings', fontweight='bold')
ax8.set_xlabel('Loading')

# 9. Performance Attribution
ax9 = plt.subplot(3, 4, 9)
attribution_effects = [
    perf_attribution['allocation_effect'],
    perf_attribution['selection_effect'],
    perf_attribution['interaction_effect']
]
effect_names = ['Allocation', 'Selection', 'Interaction']
colors = ['blue', 'green', 'orange']

bars = ax9.bar(effect_names, attribution_effects, color=colors, alpha=0.7)
ax9.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax9.set_title('Performance Attribution', fontweight='bold')
ax9.set_ylabel('Contribution (%)')

# Add value labels
for bar, value in zip(bars, attribution_effects):
    height = bar.get_height()
    ax9.text(bar.get_x() + bar.get_width()/2., height,
             f'{value:.2%}', ha='center', va='bottom' if height > 0 else 'top', fontsize=9)

# 10. Rolling Beta
ax10 = plt.subplot(3, 4, 10)
rolling_beta_data = rolling_beta['rolling_beta'].dropna()
ax10.plot(rolling_beta_data.index, rolling_beta_data, 'purple', linewidth=2)
ax10.axhline(y=1, color='black', linestyle='--', alpha=0.7, label='Beta = 1')
ax10.axhline(y=rolling_beta_data.mean(), color='red', linestyle=':', alpha=0.7, label='Average')
ax10.set_title('Rolling Beta (1-Year)', fontweight='bold')
ax10.set_ylabel('Beta')
ax10.legend(fontsize=8)
ax10.grid(True, alpha=0.3)

# 11. Returns Distribution
ax11 = plt.subplot(3, 4, 11)
ax11.hist(portfolio_returns, bins=50, alpha=0.7, color='skyblue', density=True, edgecolor='black')
ax11.axvline(portfolio_returns.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
ax11.axvline(portfolio_returns.quantile(0.05), color='orange', linestyle='--', linewidth=2, label='5% VaR')
ax11.set_title('Returns Distribution', fontweight='bold')
ax11.set_xlabel('Daily Returns')
ax11.set_ylabel('Density')
ax11.legend(fontsize=8)

# 12. Portfolio Summary
ax12 = plt.subplot(3, 4, 12)
ax12.axis('off')

# Calculate key metrics for summary
portfolio_annual_return = portfolio_returns.mean() * 252
portfolio_annual_vol = portfolio_returns.std() * np.sqrt(252)
portfolio_sharpe = (portfolio_annual_return - config.risk_free_rate) / portfolio_annual_vol
portfolio_max_dd = drawdown_analysis['max_drawdown']

summary_text = f"""
PORTFOLIO SUMMARY
═══════════════════

Annual Return:
{portfolio_annual_return:.2%}

Annual Volatility:
{portfolio_annual_vol:.2%}

Sharpe Ratio:
{portfolio_sharpe:.3f}

Maximum Drawdown:
{portfolio_max_dd:.2%}

Portfolio Beta:
{portfolio_beta['beta']:.3f}

Tracking Error:
{tracking_error_decomp['total_tracking_error']:.2%}

Information Ratio:
{risk_adj_metrics['information_ratio']:.3f}
"""

ax12.text(0.1, 0.9, summary_text, transform=ax12.transAxes, fontsize=10,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

plt.tight_layout()
plt.show()

print("📊 Comprehensive portfolio dashboard created successfully!")
print("\n🎯 Key Portfolio Insights:")
print(f"• Portfolio outperformed benchmark by {return_metrics['excess_return']:.2%} annually")
print(f"• Current portfolio has {diversification['effective_n_assets']:.1f} effective assets")
print(f"• Risk parity would improve diversification to {len(asset_returns.columns)} effective assets")
print(f"• Factor model explains {factor_exposure['r_squared']:.1%} of portfolio variance")

## Summary

This notebook demonstrated comprehensive portfolio analysis capabilities:

### ✅ **Analysis Components:**

1. **Performance Metrics** - Sharpe, Sortino, Information ratios, drawdown analysis
2. **Portfolio Construction** - Risk decomposition, return attribution, diversification analysis
3. **CAPM Analysis** - Beta calculations, security market line, multi-factor models
4. **Modern Portfolio Theory** - Efficient frontier, optimal portfolios, Black-Litterman
5. **Risk Attribution** - Factor exposure, style analysis, performance attribution
6. **Advanced Optimization** - Risk parity, resampled frontier, constraint optimization

### 🔧 **Key Features:**
- **Multi-methodology Approach**: Classical MPT, factor models, and behavioral insights
- **Risk Management**: Comprehensive risk decomposition and attribution
- **Performance Analysis**: Rolling metrics, drawdown analysis, style analysis
- **Optimization Tools**: Multiple portfolio optimization techniques
- **Professional Visualizations**: Interactive dashboards and analytical charts

### 📊 **Typical Results:**
- **Sharpe Ratios**: 0.5-1.5 for well-diversified portfolios
- **Beta Values**: 0.8-1.2 for balanced equity/bond portfolios
- **Tracking Error**: 2-6% for active strategies vs benchmarks
- **Max Drawdown**: 10-25% depending on risk level and diversification

### 🎯 **Business Applications:**
- **Portfolio Management**: Construction, optimization, and rebalancing decisions
- **Risk Management**: Risk budgeting, factor exposure monitoring
- **Performance Analysis**: Attribution analysis, benchmark comparison
- **Client Reporting**: Comprehensive performance and risk reporting
- **Investment Strategy**: Factor investing, style analysis, asset allocation

The framework provides institutional-quality portfolio analysis tools suitable for asset managers, wealth managers, pension funds, and institutional investors requiring comprehensive portfolio analytics and optimization capabilities.